In [1]:
##
## Standard approach, just scripting it
##

import numpy as np
import time

from numba import njit


s0 = 100
r =0.05
sigma = 0.3
dt = 1.0
K = 110
z = 0.1

N = 100000
x = 100

total_time = 0.0


for i in range(x):
    current_s0 = s0 + 0.0001 * i
    start_time = time.time()

    z = np.random.normal(0, 1, N)
    s = s0 * np.exp((r - sigma **2 / 2) * dt + sigma * np.sqrt(dt) * z)
    payoff = np.where(s > K, s - K, 0.0)
    discounted_payoff = np.exp(-r * dt) * np.mean(payoff) 

    end_time = time.time()
    total_time += (end_time - start_time)

average_runtime = total_time / x
print(average_runtime)

0.0018686318397521972


In [2]:
##
## Putting it in a form that calls a method
##


import numpy as np
import time
from numba import jit

s0 = 100
r = 0.05
sigma = 0.3
dt = 1.0
K = 110
z = 0.1

N = 100000
x = 100

def payoff(s0, r, sigma, dt, K, N, z):
    s = s0 * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * z)
    return np.where(s > K, s - K, 0.0)

total_time = 0.0


for i in range(x):
    
    current_s0 = s0 + 0.0001 * i
    start_time = time.time()
    z_values = np.random.normal(0, 1, N)
    average = np.exp(-r * dt) * np.mean(payoff(s0, r, sigma, dt, K, N, z_values)) 
    end_time = time.time()
    total_time += (end_time - start_time)

average_runtime = total_time / x
print(average_runtime)


0.0021969890594482422


In [3]:
##
## Having a form where jit speeds up
##


import numpy as np
import time
from numba import jit

s0 = 100
r = 0.05
sigma = 0.3
dt = 1.0
K = 110
z = 0.1

N = 100000
x = 100

@jit
def average_payoff(s0, r, sigma, dt, K, N, z):
    total_payoff = 0.0
    for _ in range(N):
        s = s0 * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * z[_])
        payoff = s - K if s > K else 0.0
        total_payoff += payoff
    return np.exp(-r * dt) * total_payoff / N

# Jit test run
z_values = np.random.normal(0, 1, N)
test_run = average_payoff(s0, r, sigma, dt, K, N, z_values) #needed to do jit compilation before run

total_time = 0.0

for i in range(x):
    current_s0 = s0 + 0.0001 * i
    start_time = time.time()

    z_values = np.random.normal(0, 1, N)
    average = average_payoff(s0, r, sigma, dt, K, N, z_values)
    
    end_time = time.time()
    total_time += (end_time - start_time)

average_runtime = total_time / x
print(average_runtime)


0.0014922070503234864
